In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('Datasets\Project 9 Heart Disease Data.csv')


data.head()


data.info()


data.describe()


In [ ]:
import seaborn as sns

corr = data.corr()

plt.figure(figsize = (15,15))
sns.heatmap(corr, annot = True)

In [ ]:
corr


In [ ]:
sns.set_style('whitegrid')
sns.countplot(x = 'target', data = data)

In [ ]:
# dataset = pd.get_dummies(data, columns = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'])


In [ ]:
dataset = data.copy()
dataset.head()

In [ ]:
X = dataset.drop(['target'], axis = 1)
y = dataset['target']

In [ ]:
X.columns


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)
pred[:10]

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
print(f"Accuracy of model is {round(accuracy_score(y_test, pred)*100, 2)}%")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
classifier = RandomForestClassifier(n_jobs = -1)


In [ ]:
from scipy.stats import randint
param_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200,300,400,500],
              'max_features':randint(1,31),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,31),
              }

In [ ]:
search_clfr = RandomizedSearchCV(classifier, param_distributions = param_dist, n_jobs=-1, n_iter = 40, cv = 9)


In [ ]:
search_clfr.fit(X_train, y_train)


In [ ]:
params = search_clfr.best_params_
score = search_clfr.best_score_
print(params)
print(score)

In [ ]:
claasifier=RandomForestClassifier(n_jobs=-1, n_estimators=400,bootstrap= False,criterion='gini',max_depth=5,max_features=3,min_samples_leaf= 7)


In [ ]:
classifier.fit(X_train, y_train)


In [ ]:
RandomForestClassifier(n_jobs=-1)


In [ ]:
confusion_matrix(y_test, classifier.predict(X_test))


In [ ]:
print(f"Accuracy is {round(accuracy_score(y_test, classifier.predict(X_test))*100,2)}%")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Load dataset (replace with actual dataset path or URL)
url = "/content/Project 9 Heart Disease Data.csv"
df = pd.read_csv(url)

# Split dataset into features (X) and target (y)
X = df.drop(columns=['target'])  # Assuming 'target' is the label
y = df['target']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize models
models = {
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=500, random_state=42)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred)
    results[name] = accuracy * 100
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")
    print(f"{name} AUC-ROC: {auc_roc:.2f}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

# Check for Overfitting using Cross-Validation
for name, model in models.items():
    cv_scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} Cross-validation Accuracy: {np.mean(cv_scores) * 100:.2f}%")

# Plot Model Performance
plt.figure(figsize=(8,5))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.ylabel("Accuracy (%)")
plt.title("Model Comparison")
plt.show()


In [ ]:
import pickle
pickle.dump(classifier, open('heart.pkl', 'wb'))